In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
import scanpy as sc
import anndata as ad
import scvelo as scv
import scvi
import seaborn as sns
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output,dash_table, ctx

import pandas as pd

import os
import sys
import time
import warnings
import gc
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.lines import Line2D 

from copy import copy
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")

project_directory = '/Cranio_Lab/Louk_Seton/4_species_project'
os.chdir(os.path.expanduser("~")+project_directory)

In [ ]:
adata = sc.read('h5ad_files/mouse/ecto_andrea/ecto_combined_all_integrated.h5ad')

In [ ]:
adata

In [ ]:
sc.pl.umap(adata,color =['leiden_high'])

In [ ]:
scv.pl.proportions(adata,groupby = 'leiden_high',figsize =  (10,8))

In [ ]:
warnings.filterwarnings('ignore')

scv.pp.moments(adata, use_rep = 'X_umap_features',
               #use_highly_variable=True,
               n_neighbors=15)
scv.tl.velocity(adata)
scv.tl.velocity_graph(adata,n_jobs = 10)


In [ ]:
warnings.filterwarnings('ignore')

scv.pl.velocity_embedding_stream(adata, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,)

In [ ]:
warnings.filterwarnings('ignore')

scv.pl.velocity_embedding_stream(adata, color = 'leiden_high',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,)

In [ ]:
warnings.filterwarnings('ignore')

scv.pl.velocity_embedding_stream(adata, color = 'annotation_ecto',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,)

In [ ]:
leiden_res = [0.5,0.6,0.7,0.8,0.9,1,1.2,1.4,1.6,1.8,2,2.2,2.4,2.6,2.8,3]
for i in leiden_res:
    sc.tl.leiden(adata, resolution = i, key_added = 'leiden'+str(i))
try:
    del adata.uns['log1p']
except: print('no log1p')
leiden_res = [0.5,0.6,0.7,0.8,0.9,1,1.2,1.4,1.6,1.8,2,2.2,2.4,2.6,2.8,3]
for i in leiden_res:
    sc.tl.rank_genes_groups(adata, 'leiden'+str(i), method='wilcoxon', 
                            key_added = "wilcoxon_"+'leiden'+str(i))

sc.tl.rank_genes_groups(adata, 'leiden_high', method='wilcoxon', 
                            key_added = "wilcoxon_"+'leiden_high')

In [ ]:
def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k * h)[:3]) * 255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

plotly_reds = matplotlib_to_plotly(reds, 255)
plotly_reds[0] = [0.0, 'rgb(211, 211, 211)']

In [ ]:
adata.var['gene_symbol'] = adata.var['gene_name']

In [ ]:
adata.write('h5ad_files/mouse/ecto_andrea/ecto_interactive.h5ad')

In [ ]:
#Interactive dash app to view 3D plot
#create quick dataframe
tmp_df = pd.DataFrame(adata.obsm['X_umap'], columns = ['X','Y'])
tmp_df.index = adata.obs.index
tmp_df = tmp_df.join(adata.obs)
tmp_df['barcode'] = list(tmp_df.index) 
port = 11188
app = Dash(__name__)


app.layout = html.Div([
    html.Div([
        html.Div([
            html.Label("Select gene"),
            dcc.Dropdown(adata.var.index.tolist(), 
                         value='Sox10',
                         id="dropdown_var",
                        ),
            html.Label(id='gene_id'),
            html.Br(),
            html.Label(id='gene_product'),
            html.Br(),
            html.Div([],id='db_xref'),
            html.Br(),
            html.Label("Point size"),
            dcc.Slider(1, 10,
                       marks = None,
                       value=2,
                       id='slider_marker_size',
                       tooltip={"placement": "bottom", "always_visible": True},
                      ),
        ],
            style={'width': '49%', 'display': 'inline-block'}),
        html.Div([
            html.Label("Select obs"),
            dcc.Dropdown(adata.obs.columns.tolist(), 
                         value='leiden_high',
                         id="dropdown_obs",
                        ),
            
        ],
           style={'width': '49%', 'float': 'right', 'display': 'inline-block'}),
    ],
        style={'padding': '10px 5px'}),
    html.Div([
      dcc.Graph(id="graph1"),
        
    ],
        style={'width': '49%', 'display': 'inline-block'}),
     html.Div([
      dcc.Graph(id="graph2")
    ],
        style={'width': '49%', 'display': 'inline-block'}),
    html.Div(id="cluster_select",
        style={'width': '49%', 'display': 'inline-block'},),
    html.Div(id='deg_table'),
    html.Div(dcc.Store(id='gene_plotted')),
])
    
                
            

@app.callback(
    Output("graph1", "figure"),
    Output('gene_plotted','data'),
    Input("slider_marker_size", "value"),
    Input('deg_table_callback', 'active_cell'),
    Input("dropdown_var", "value"),
    Input('deg_table_callback', 'derived_viewport_data'),
)
def update_graph_1(size_var,active_cell,dropdown_var,data_deg):
    callback_id = ctx.triggered_id if not None else 'dropdown_var'
    if callback_id == 'dropdown_var':
        color_var = dropdown_var
    elif callback_id == 'deg_table_callback':
        try:
            color_var = data_deg[active_cell['row']]['names']
        except:
            color_var = dropdown_var
        
    else:
        color_var = dropdown_var
    plot_df = tmp_df
    fig = px.scatter(plot_df, x='X', y='Y',
                        color = adata[:,color_var].X.toarray().flatten(),
                        hover_data={'X':False,
                                    'Y':False,
                                    'barcode':True,
                                    'stage':True,
                                    'phase':True,},
                        color_continuous_scale = plotly_reds,
                        labels = {'color':color_var},
                        width=700, height=700,
                       )
    fig.update_traces(marker={'size': size_var,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                         })
    fig.update_layout(uirevision='constant')
    return fig, color_var

@app.callback(
    Output("graph2", "figure"),
    Input("dropdown_obs", "value"),
    Input("slider_marker_size", "value"),
)
def update_graph_2(color_var, size_var):
    plot_df = tmp_df
    sorted_list = []
    for i in plot_df:
        if pd.api.types.is_categorical_dtype(plot_df[i]):
            plot_df[i] = plot_df[i].cat.remove_unused_categories()
    if pd.api.types.is_categorical_dtype(plot_df[color_var]):
        try:
            sorted_list = sorted(list(plot_df[color_var].unique().categories), key = int)
        except:
            sorted_list = sorted(list(plot_df[color_var].unique().categories))
                             
     
    fig = px.scatter(plot_df, x='X', y='Y',
                        color = plot_df[color_var],
                        hover_data={'X':False,
                                    'Y':False,
                                    'barcode':True,
                                    'stage':True,
                                    'phase':True,},
                        #color='leiden_0.5',
                        color_continuous_scale = plotly_reds,
                        category_orders={color_var: sorted_list},
                        width=800, height=700,
                       )
    
    fig.update_traces(marker={'size': size_var,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                         })
    fig.update_layout(legend= {'itemsizing': 'constant'})

    fig.update_layout(uirevision='constant')
    return fig


@app.callback(
    Output("gene_id", "children"),
    Input("gene_plotted", "data"),
)
def update_gene_id(gene_selected):
    gene_id = adata.var.loc[gene_selected,'gene_symbol']
    return f'{gene_id}'



@app.callback(Output('cluster_select','children'),
              Input('dropdown_obs','value'),
             )
def cluster_dropdown(leiden_res):
    if 'leiden' in leiden_res or 'annotation_paper_coarse' in leiden_res or 'annotation_2024' in leiden_res:
        return dcc.Dropdown(list(adata.obs[leiden_res].cat.categories), 
                           value=list(adata.obs[leiden_res].cat.categories)[0],id = 'cluster_select2')

@app.callback(Output('deg_table','children'),
              Input('dropdown_obs','value'),
              Input('cluster_select2','value'),
             )
def update_datatable(leiden_res,cluster_select):            
    if 'leiden' in leiden_res or 'annotation_paper_coarse' in leiden_res or 'annotation_2024' in leiden_res:
        deg_df = sc.get.rank_genes_groups_df(adata, group=None, key='wilcoxon_'+leiden_res)
        deg_df = deg_df[deg_df['group']==cluster_select].iloc[:,1:6]
        data_deg = deg_df.to_dict('records')
        columns =  [{"name": i, "id": i,} for i in (deg_df.columns)]
        return dash_table.DataTable(data=data_deg, 
                                    columns=columns,
                                    filter_action="native",
                                    sort_action="native",
                                    sort_mode="multi",
                                    column_selectable="single",
                                    page_action="native",
                                    page_current= 0,
                                    page_size= 10,
                                    id = 'deg_table_callback',
                                   )

app.run(jupyter_mode="tab",host='0.0.0.0', port = port ,debug=False)
print('Use this link: '+'http://cranio213:'+str(port))